In [ ]:
!pip install scikeras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import glob
from sklearn.preprocessing import StandardScaler
from keras.layers import Dropout
from keras.regularizers import L1L2
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from sklearn.utils import class_weight
from sklearn import preprocessing
from keras.utils import to_categorical
from keras import metrics
from collections import Counter
from keras import regularizers
from keras.callbacks import EarlyStopping, ModelCheckpoint
import numpy as np
import tensorflow as tf
from keras.layers import LSTM
from sklearn.model_selection import StratifiedKFold, cross_val_score
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import KFold
from scikeras.wrappers import KerasClassifier
from keras.utils import to_categorical
np.random.seed(0)
tf.random.set_seed(0)

In [ ]:
# load all csv files
path = '/content/drive/MyDrive/Data/HRV/1_5' # use your path
all_files = glob.glob(path + "/*.csv")


li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

df = pd.concat(li, axis=0, ignore_index=True)

# separate into input (X) and output (y) variables
X = df[['RMSSD','HF','LF']].values
y = df['Label'].values

# encode class values as integers
y = np_utils.to_categorical(y)

# split into 80% for training and 20% for testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Revert one-hot-encoded y_train back to class labels
y_train_labels = np.argmax(y_train, axis=1)

# calculate class weights
class_weights = class_weight.compute_sample_weight('balanced', y_train_labels)

# Create class weight dictionary
weights = dict(enumerate(class_weights))


# li = []

# for filename in all_files:
#     df = pd.read_csv(filename, index_col=None, header=0)
#     li.append(df)

# df = pd.concat(li, axis=0, ignore_index=True)

# # separate into input (X) and output (y) variables
# X = df[['RMSSD','HF','LF']].values
# y = df['Label'].values

# # split into 80% for training and 20% for testing
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # # encode class values as integers
# # y = np_utils.to_categorical(y)

# # feature scaling
# scaler = StandardScaler()
# X = scaler.fit_transform(X)

# # Revert one-hot-encoded y_train back to class labels
# y_train_labels = np.argmax(y_train, axis=0)

# # calculate class weights
# class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train_labels), y_train_labels)

# # Convert class weights to dictionary to use with Keras
# class_weights_dict = dict(enumerate(class_weights))

# define the keras model
model = Sequential()
model.add(Dense(64, input_dim=3, activation='tanh', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dropout(0.05))
model.add(Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dropout(0.05))
model.add(Dense(16, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dropout(0.05))
model.add(Dense(8, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dropout(0.05))
model.add(Dense(4, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dropout(0.05))
model.add(Dense(3, activation='softmax'))

# compile the keras model
opt = Adam(learning_rate=0.001)
# RMSprop optimizer
opt_rms = keras.optimizers.RMSprop(learning_rate=0.001)
# Nadam optimizer
opt_nadam = keras.optimizers.Nadam(learning_rate=0.001)
# Adadelta optimizer
opt_adadelta = keras.optimizers.Adadelta(learning_rate=1.0)

# Adagrad optimizer
opt_adagrad = keras.optimizers.Adagrad(learning_rate=0.01)

# Adamax optimizer
opt_adamax = keras.optimizers.Adamax(learning_rate=0.002)

#2 Best adagrad with early stop 95.76
#1 Best adamax with early stop 96.61
#The best HOW IT IS RIGHT NOW. Accuracy: 97.46

model.compile(loss='categorical_crossentropy', optimizer=opt_adamax, metrics=['accuracy'])

# define early stopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)

# # define model checkpointing
mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

# fit the keras model on the dataset
model.fit(X_train, y_train, validation_split=0.2, epochs=150, batch_size=20, verbose=1, callbacks=[es, mc])

# fit the keras model on the dataset
# model.fit(X_train, y_train, validation_split=0.2, epochs=150, batch_size=10, verbose=1, class_weight=weights)

# model = Sequential()
# model.add(LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2])))
# model.add(Dense(3, activation='softmax')) # Change according to number of classes
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# # Fit network
# history = model.fit(X_train, y_train, epochs=50, batch_size=72, verbose=1, validation_split=0.2)

# evaluate the keras model
_, accuracy = model.evaluate(X_test, y_test, verbose=0)
print('Accuracy: %.2f' % (accuracy*100))

# make class predictions with the model
probabilities = model.predict(X_test)
predictions = np.argmax(probabilities, axis=-1)

# summarize the first 5 cases
for i in range(5):
    print('%s => %d (expected %d)' % (X_test[i].tolist(), predictions[i], np.argmax(y_test[i])))

# confusion matrix
print(confusion_matrix(np.argmax(y_test, axis=1), predictions))

# classification report
print(classification_report(np.argmax(y_test, axis=1), predictions))

Epoch 1/150
 1/19 [>.............................] - ETA: 13s - loss: 1.8898 - accuracy: 0.9000
Epoch 1: val_accuracy improved from -inf to 0.68421, saving model to best_model.h5
19/19 [==============================] - 1s 12ms/step - loss: 1.8091 - accuracy: 0.7613 - val_loss: 1.7846 - val_accuracy: 0.6842
Epoch 2/150
 1/19 [>.............................] - ETA: 0s - loss: 1.7429 - accuracy: 0.7000
Epoch 2: val_accuracy did not improve from 0.68421
19/19 [==============================] - 0s 4ms/step - loss: 1.6318 - accuracy: 0.7772 - val_loss: 1.6493 - val_accuracy: 0.6842
Epoch 3/150
 1/19 [>.............................] - ETA: 0s - loss: 1.6423 - accuracy: 0.7000
Epoch 3: val_accuracy did not improve from 0.68421
19/19 [==============================] - 0s 3ms/step - loss: 1.4685 - accuracy: 0.7719 - val_loss: 1.5512 - val_accuracy: 0.6842
Epoch 4/150
 1/19 [>.............................] - ETA: 0s - loss: 1.4706 - accuracy: 0.7000
Epoch 4: val_accuracy did not improve from 0.6

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# load all csv files
path = '/content/drive/MyDrive/Data/HRV/1_5' # use your path
all_files = glob.glob(path + "/*.csv")


li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

df = pd.concat(li, axis=0, ignore_index=True)

# separate into input (X) and output (y) variables
X = df[['RMSSD','HF','LF']].values
y = df['Label'].values

# encode class values as integers
y = np_utils.to_categorical(y)

# split into 80% for training and 20% for testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Revert one-hot-encoded y_train back to class labels
y_train_labels = np.argmax(y_train, axis=1)
# reshape from [samples, features] into [samples, timesteps, features]
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

model = Sequential()
model.add(LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2]), activation='relu'))
model.add(Dense(32, activation='relu',kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=5)
model.fit(X_train, y_train, epochs=150, batch_size=10, verbose=1, validation_split=0.3, callbacks=[early_stopping])

# Fit network
#history = model.fit(X_train, y_train, epochs=150, batch_size=10, verbose=1, validation_split=0.3)

# evaluate the keras model
_, accuracy = model.evaluate(X_test, y_test, verbose=0)
print('Accuracy: %.2f' % (accuracy*100))

# make class predictions with the model
probabilities = model.predict(X_test)
predictions = np.argmax(probabilities, axis=-1)

# summarize the first 5 cases
for i in range(5):
    print('%s => %d (expected %d)' % (X_test[i].tolist(), predictions[i], np.argmax(y_test[i])))

# confusion matrix
print(confusion_matrix(np.argmax(y_test, axis=1), predictions))

# classification report
print(classification_report(np.argmax(y_test, axis=1), predictions))


Epoch 1/150
29/29 [==============================] - 2s 14ms/step - loss: 1.3938 - accuracy: 0.6747 - val_loss: 1.0772 - val_accuracy: 0.7258
Epoch 2/150
29/29 [==============================] - 0s 3ms/step - loss: 0.8717 - accuracy: 0.7785 - val_loss: 1.0511 - val_accuracy: 0.7339
Epoch 3/150
29/29 [==============================] - 0s 4ms/step - loss: 0.8219 - accuracy: 0.8201 - val_loss: 0.8913 - val_accuracy: 0.8387
Epoch 4/150
29/29 [==============================] - 0s 6ms/step - loss: 0.8185 - accuracy: 0.7958 - val_loss: 0.6897 - val_accuracy: 0.8548
Epoch 5/150
29/29 [==============================] - 0s 4ms/step - loss: 0.7850 - accuracy: 0.8512 - val_loss: 0.8442 - val_accuracy: 0.8306
Epoch 6/150
29/29 [==============================] - 0s 4ms/step - loss: 0.7562 - accuracy: 0.8685 - val_loss: 0.6588 - val_accuracy: 0.8226
Epoch 7/150
29/29 [==============================] - 0s 4ms/step - loss: 0.5498 - accuracy: 0.8201 - val_loss: 0.6836 - val_accuracy: 0.7903
Epoch 8/150


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
